## Programming Assignment - Recommending Songs

In [2]:
import turicreate as tc

In [7]:
songs = tc.SFrame('song_data.sframe')
songs.head()

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


### 1. Counting unique users
Calculate the number of unique users for 'Kanye West', 'Foo Fighters', 'Taylor Swift' and 'Lady GaGa'.

In [12]:
for artist in ['Kanye West', 'Foo Fighters', 'Taylor Swift', 'Lady GaGa']:
    artist_songs = songs[songs['artist'] == artist]
    print('Num unique users for {artist}: {unique_users}'.format(artist=artist,
                                                                 unique_users=len(artist_songs['user_id'].unique())))

Num unique users for Kanye West: 2522
Num unique users for Foo Fighters: 2055
Num unique users for Taylor Swift: 3246
Num unique users for Lady GaGa: 2928


### **2. Find the most and least popular artist by listen count**

In [16]:
artists = songs.groupby(key_column_names='artist',
                        operations = {'total_count': tc.aggregate.SUM('listen_count')})
artists.head()

artist,total_count
Michael Nyman,255
Bayside,411
Aaliyah,1566
Parkway Drive,203
Bitty McLean,535
Bring Me The Horizon,1112
Bullet For My Valentine,3620
Boom Bip,140
Delta Spirit,98
Groove Armada,631


In [19]:
# most popular artist
artists.sort('total_count', ascending=False).head(1)['artist']

dtype: str
Rows: 1
['Kings Of Leon']

In [20]:
# least popular artist
artists.sort('total_count', ascending=True).head(1)['artist']

dtype: str
Rows: 1
['William Tabbert']

### **3. Find the most recommended songs**

Find the song most recommeded by the trained personalised_model.

In [29]:
train_data, test_data = songs.random_split(.8, seed=0)
print(len(train_data))
print(len(test_data))

893580
223029


In [26]:
personalised_model = tc.item_similarity_recommender.create(train_data, user_id = 'user_id', item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 2.90s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.28552s

In [27]:
# use only the first 10,000 users from the test data
subset_test_users = test_data['user_id'].unique()[:10000]

In [30]:
# find one recommended song for each user
recommendations = personalised_model.recommend(subset_test_users, k=1)
recommendations

recommendations finished on 1000/10000 queries. users per second: 92395.8

recommendations finished on 2000/10000 queries. users per second: 18517.3

recommendations finished on 3000/10000 queries. users per second: 14571.7

recommendations finished on 4000/10000 queries. users per second: 18804.5

recommendations finished on 5000/10000 queries. users per second: 16139.4

...

recommendations finished on 7000/10000 queries. users per second: 13808.9

recommendations finished on 8000/10000 queries. users per second: 15551.7

recommendations finished on 9000/10000 queries. users per second: 14683

recommendations finished on 10000/10000 queries. users per second: 13973

user_id,song,score,rank
18fafad477f9d72ff86f7d0bd838a6573de0f64a ...,Rabbit Heart (Raise ItUp) - Florence + The ...,0.07993997260928154,1
7c5ab90ba508502a888843a71eef80bf186c7b88 ...,Hypnopaedia - OctopusProject ...,0.030612250169118244,1
974ed1d7e1b2253787987a36a40ea6e74f893238 ...,Secrets - OneRepublic,0.03843166306614876,1
f9d2599a999d601ad598fe0cb79b41329ba3d42c ...,Hello - Lionel Richie,0.024615383148193358,1
8e371e084913c4cc2f5bb48d5070df25d53369a2 ...,Costruire - Niccolò Fabi,0.030949632326761883,1
c8cf3b33060579e75142ad4d3803f0b4c7ad921f ...,Sunspots - Nine InchNails ...,0.026633880640331068,1
251f602d5c644925ac89b840eb1a22d9026bde18 ...,Ten Cent Pistol - TheBlack Keys ...,0.07262169073025386,1
0bac31bb00006b63cab862efe85c3e92ce604147 ...,Runnin' With The Devil -Van Halen ...,0.05460588137308756,1
e238e498000d3f678d2cd4085eddc256b9d7ed58 ...,Stranger - Angus & JuliaStone ...,0.03459119300047556,1
1c7e130c6dc23c133d8966f81ca55dfe8316d78f ...,West One (Shine On Me) -The Ruts ...,0.014672351734978812,1


In [ ]:
# count how many times each song has been recommended
recommendation_counts = recommendations.groupby('song', operations={'count': tc.aggregate.COUNT()})
# sort the results in descending order
recommendation_counts.sort('count', ascending = False).head()